In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
import matplotlib.pyplot as plt
from torchsummary import summary
import numpy as np

import torchvision
import torchvision.transforms as transforms
from tqdm import trange

In [2]:
# Hyperparameters for training
num_rounds = 200
lr = 0.001
num_channels = 3
num_classes = 10
test_bs = 128

In [3]:
device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')

In [4]:
device

device(type='cuda', index=1)

In [5]:
class ResNet(nn.Module):
    def __init__(self, block, num_blocks, in_channels=1, num_classes=2):
        super(ResNet, self).__init__()
        self.in_planes = 64

        self.conv1 = nn.Conv2d(in_channels, 64, kernel_size=3,
                               stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.linear = nn.Linear(512 * block.expansion, num_classes)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = self.avgpool(out)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out

In [6]:
class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(
            in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)

        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3,
                               stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out

In [7]:
def ResNet18(in_channels, num_classes):
    return ResNet(BasicBlock, [2, 2, 2, 2], in_channels=in_channels, num_classes=num_classes)

In [8]:
class DatasetSplit(Dataset):
    def __init__(self, dataset, idxs):
        self.dataset = dataset
        self.targets = dataset.targets
        self.idxs = list(idxs)

    def __len__(self):
        return len(self.idxs)

    def __getitem__(self, item):
        image, label = self.dataset[self.idxs[item]]
        return image, label

In [9]:
# transform_train = transforms.Compose([transforms.ToTensor(),
#                                       transforms.RandomCrop(32, padding=4),
#                                       transforms.RandomHorizontalFlip(),
#                                       transforms.Normalize(mean=[x / 255.0 for x in [125.3, 123.0, 113.9]],
#                                      std=[x / 255.0 for x in [63.0, 62.1, 66.7]])])
# transform_test = transforms.Compose([
#     transforms.ToTensor(),
#     transforms.Normalize(mean=[x / 255.0 for x in [125.3, 123.0, 113.9]], std=[x / 255.0 for x in [63.0, 62.1, 66.7]])])

In [10]:
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

In [12]:
trainset = torchvision.datasets.CIFAR10(
    root='./data', train=True, download=True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(
    trainset, batch_size=512, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(
    root='./data', train=False, download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(
    testset, batch_size=test_bs, shuffle=False, num_workers=2)

Files already downloaded and verified
Files already downloaded and verified


In [13]:
# Initialize and synchronize models###
global_model = ResNet18(in_channels=num_channels, num_classes=num_classes).to(device)
# optimizer = torch.optim.SGD(global_model.parameters(), lr=lr, momentum=0.9, weight_decay=5e-4)
optimizer = torch.optim.Adam(global_model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=200)
# scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=milestones, gamma=gamma)
criterion = nn.CrossEntropyLoss()
              
# client_models = [ResNet18(
#     in_channels=num_channels, num_classes=num_classes).to(device) for _ in range(num_clients)]
# optimizers=[torch.optim.Adam(model.parameters(), lr=lr) for model in client_models]
# schedulers = [torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=milestones, gamma=gamma) for optimizer in optimizers]
# for model in client_models:
#   model.load_state_dict(global_model.state_dict())

In [14]:
# summary(global_model, (3, 32, 32))

In [15]:
# Training function
def train(model: nn.Module, loader: DataLoader, optimizer, criterion):
    model.train()
    epoch_loss = []
    epoch_acc = []
    for batch_idx, (data, target) in enumerate(loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        epoch_loss.append(loss.item())

        _, pred = torch.max(output, dim=1)
        correct_train = pred.eq(target.data.view_as(pred))
        accuracy_train = torch.mean(correct_train.type(torch.FloatTensor))
        epoch_acc.append(accuracy_train.item())
    return model.state_dict(), sum(epoch_loss) / len(epoch_loss), sum(epoch_acc) / len(epoch_acc)

In [16]:
# Evaluate on Test dataset
def evaluate(global_model, data_loader, device):
    global_model.eval()
    # testing
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in data_loader:
            data, target = data.to(device), target.to(device)
            log_probs = global_model(data)
            # sum up batch loss
            test_loss += F.cross_entropy(log_probs,
                                         target, reduction='sum').item()
            # get the index of the max log-probability
            y_pred = log_probs.data.max(1, keepdim=True)[1]
            correct += y_pred.eq(target.data.view_as(y_pred)
                                 ).long().cpu().sum()

        test_loss /= len(data_loader.dataset)
        accuracy = 100.00 * correct.item() / len(data_loader.dataset)
    return accuracy, test_loss

In [17]:
global_model.train()
loss_train = []
loss_test = []
acc_train = []
acc_test= []
# Early stopping
patience = 200 
patience_counter = 0


for round in trange(num_rounds):
    # Train on local data
    w, loss, acc = train(global_model, trainloader, optimizer, criterion)
    loss_train.append(loss)
    acc_train.append(100 * acc)

    # Evaluation on test dataset for each round
    cur_auc, test_loss = evaluate(global_model, testloader, device)
    loss_test.append(test_loss)
    acc_test.append(cur_auc)
    print(f'Round {round+1}, Train loss: {loss:.4f}, Test Loss: {test_loss:.4f}, Test Accuracy: {cur_auc:.2f}%')
     ## Early Stopping
    valid_loss_array = np.array(loss_test)
    min_valid_loss = np.min(valid_loss_array)
    
    if(test_loss > min_valid_loss):
        patience_counter += 1        
    else:
        patience_counter = 0       #setting the patience counter to zero if the test loss improves again 
    
    ## Calling early stopping if test loss doest improves from last (patience) Iterations
    if(patience_counter > patience):
        print("Early stopping called at {} epochs".format(round+1))
        break
    scheduler.step()


  0%|          | 1/200 [00:11<38:16, 11.54s/it]

Round 1, Train loss: 1.6268, Test Loss: 1.7896, Test Accuracy: 38.37%


  1%|          | 2/200 [00:22<37:29, 11.36s/it]

Round 2, Train loss: 1.1139, Test Loss: 1.1533, Test Accuracy: 59.89%


  2%|▏         | 3/200 [00:34<37:10, 11.32s/it]

...

Round 198, Train loss: 0.0001, Test Loss: 0.5506, Test Accuracy: 93.23%


100%|█████████▉| 199/200 [38:02<00:11, 11.46s/it]

Round 199, Train loss: 0.0001, Test Loss: 0.5519, Test Accuracy: 93.27%


100%|██████████| 200/200 [38:14<00:00, 11.47s/it]

Round 200, Train loss: 0.0001, Test Loss: 0.5479, Test Accuracy: 93.21%


In [ ]:
# Testing
global_model.eval()
train_acc, train_loss = evaluate(
    global_model, trainloader, device)
test_acc, test_loss = evaluate(
    global_model, testloader, device)
print("Training accuracy: {:.2f}%".format(train_acc))
print("Testing accuracy: {:.2f}%".format(test_acc))

In [18]:
# Save the entire model
torch.save(global_model, 'central_model_resnet18.pth')

# # To load the entire model
# model = torch.load('model_complete.pth')
# model.eval()

In [ ]:
with open(f'Central_acc_train.txt', 'w') as file:
    # Write each value on a new line
    file.write(f"{acc_train}\n")
with open(f'Central_acc_test.txt', 'w') as file:
    # Write each value on a new line
    file.write(f"{acc_test}\n")
with open(f'Central_loss_train.txt', 'w') as file:
    # Write each value on a new line
    file.write(f"{loss_train}\n") 
with open(f'Central_loss_test.txt', 'w') as file:
    # Write each value on a new line
    file.write(f"{loss_test}\n")